In [42]:
import pandas as pd #DataFrame

from sklearn.preprocessing import LabelEncoder #Encoding our classes (Mines and Rocks)
from sklearn.model_selection import train_test_split, GridSearchCV # Splitting the data for training and testing. GridSearchCV for finding the best combination
from sklearn.model_selection import StratifiedKFold # Stratified K-Fold Cross Validation for stratified sampling
from sklearn.linear_model import LogisticRegression # Model 1
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, classification_report # For accuracy reports

from imblearn.over_sampling import SMOTE # Over/Under sampling the dataset
import xgboost as xgb # Model 2

In [43]:
df = pd.read_csv(r'E:\datasets\ml\sonar.all-data.csv', header=None)
type(df)
df.head()
df.shape

(208, 61)

In [44]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
count,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,...,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000
mean,0.029164,0.038437,0.043832,0.053892,0.075202,0.104570,0.121747,0.134799,0.178003,0.208259,...,0.016069,0.013420,0.010709,0.010941,0.009290,0.008222,0.007820,0.007949,0.007941,0.006507
std,0.022991,0.032960,0.038428,0.046528,0.055552,0.059105,0.061788,0.085152,0.118387,0.134416,...,0.012008,0.009634,0.007060,0.007301,0.007088,0.005736,0.005785,0.006470,0.006181,0.005031
min,0.001500,0.000600,0.001500,0.005800,0.006700,0.010200,0.003300,0.005500,0.007500,0.011300,...,0.000000,0.000800,0.000500,0.001000,0.000600,0.000400,0.000300,0.000300,0.000100,0.000600
25%,0.013350,0.016450,0.018950,0.024375,0.038050,0.067025,0.080900,0.080425,0.097025,0.111275,...,0.008425,0.007275,0.005075,0.005375,0.004150,0.004400,0.003700,0.003600,0.003675,0.003100
50%,0.022800,0.030800,0.034300,0.044050,0.062500,0.092150,0.106950,0.112100,0.152250,0.182400,...,0.013900,0.011400,0.009550,0.009300,0.007500,0.006850,0.005950,0.005800,0.006400,0.005300
75%,0.035550,0.047950,0.057950,0.064500,0.100275,0.134125,0.154000,0.169600,0.233425,0.268700,...,0.020825,0.016725,0.014900,0.014500,0.012100,0.010575,0.010425,0.010350,0.010325,0.008525
max,0.137100,0.233900,0.305900,0.426400,0.401000,0.382300,0.372900,0.459000,0.682800,0.710600,...,0.100400,0.070900,0.039000,0.035200,0.044700,0.039400,0.035500,0.044000,0.036400,0.043900


In [45]:
df[60].value_counts()

60
M    111
R     97
Name: count, dtype: int64

In [46]:
# Preprocessing the data
df.dropna(inplace=True)
X = df.drop(columns=60, axis=1)
Y = df[60]

In [47]:
# Split the dataset into training and testing, 80% for training and 20% for testing, set random state to prevent randomness across different runs
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)

### Logistic Regression

In [48]:
# Load logistic Regression
model = LogisticRegression()

In [49]:
# Fitting the data into the model, train it
model.fit(X_train, Y_train)

LogisticRegression()

In [50]:
# Feed in the training data and fetch the accuracy score
training_predict = model.predict(X_train)
training_accuracy = accuracy_score(training_predict, Y_train)

In [51]:
# Print the accuracy on the training dataset
print(f"accuracy when tested with training data = {training_accuracy}")

accuracy when tested with training data = 0.8373493975903614


In [62]:
# Print the classification report
Y_pred_lr = model.predict(X_test)
print(classification_report(Y_test, Y_pred_lr))

              precision    recall  f1-score   support

           M       0.81      0.95      0.88        22
           R       0.94      0.75      0.83        20

    accuracy                           0.86        42
   macro avg       0.87      0.85      0.85        42
weighted avg       0.87      0.86      0.86        42



### Random Forest Classifier

In [52]:
# Import SMOTE with a fixed random state for consistent results across different runs  
sm = SMOTE(random_state=42)

# Apply SMOTE to training data for balanced learning (we have more rocks than mines in the dataset)
X_train_resampled, Y_train_resampled = sm.fit_resample(X_train, Y_train)

In [54]:
# Define hyperparameters for Random Forest
param_grid_rf = {
    'n_estimators': [100, 200, 225, 250, 300],
    'max_depth': [3, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Load the RFC model
rf_model = RandomForestClassifier(random_state=42)

# To prevent random sampling which would result in otherwise bad accuracy, here, recall
stratified_cv = StratifiedKFold(n_splits=10)

# Initialize GridSearchCV with Random Forest and resampled data
grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid_rf, cv=stratified_cv, n_jobs=-1, verbose=2)

# Fit the resampled data and train the model
grid_search_rf.fit(X_train_resampled, Y_train_resampled)

Fitting 10 folds for each of 180 candidates, totalling 1800 fits


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [3, 5, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 225, 250, 300]},
             verbose=2)

In [55]:
# Fetch the best params and recall score
best_params_rf = grid_search_rf.best_params_
best_recall_rf = grid_search_rf.best_score_

# Print the params and recall score
print(f"Best parameters (RF): {best_params_rf}")
print(f"Best recall score (RF): {best_recall_rf}")

# Generate the classification report
best_rf_model = grid_search_rf.best_estimator_
Y_pred_rf = best_rf_model.predict(X_test)
print(classification_report(Y_test, Y_pred_rf))

Best parameters (RF): {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best recall score (RF): 0.888888888888889
              precision    recall  f1-score   support

           M       0.81      1.00      0.90        22
           R       1.00      0.75      0.86        20

    accuracy                           0.88        42
   macro avg       0.91      0.88      0.88        42
weighted avg       0.90      0.88      0.88        42



### XGBoost

In [56]:
# Import SMOTE with a fixed random state for consistent results across different runs  
sm = SMOTE(random_state=42)

# Apply SMOTE to training data for balanced learning (we have more rocks than mines in the dataset)
X_train_resampled, Y_train_resampled = sm.fit_resample(X_train, Y_train)

In [57]:
# Encode classes (M and R)
label_encoder = LabelEncoder()
Y_train_resampled_encoded = label_encoder.fit_transform(Y_train_resampled)
Y_test_encoded = label_encoder.transform(Y_test)

In [58]:
# Print the encoded classes for sanity
for i, class_label in enumerate(label_encoder.classes_):
    print(f"Class {i}: {class_label}")

Class 0: M
Class 1: R


In [63]:
# Init XGBoost
xgb_model = xgb.XGBClassifier(use_label_encoder=False,
                              eval_metric='logloss', # logloss is well suited for classification problems
                              random_state=42)

# Hyperparameters
param_grid_xgb = {
    'n_estimators': [100, 200, 225, 250, 300],
    'max_depth': [3, 5, 10, 20],
    'learning_rate': [0.01, 0.05, 0.1],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.6, 0.8, 1.0],
    'alpha' : [0.0, 0.05, 0.1, 1.0], # L1 regularizer (lasso)
    'lambda' : [0.0, 0.1, 1.0, 5.0, 10.0] # L2 regularizer (Ridge)
}

# To prevent random sampling which would result in otherwise bad accuracy, here, recall
stratified_cv = StratifiedKFold(n_splits=10)

# Initialize GridSearchCV with resampled data and params
grid_search_xgb = GridSearchCV(estimator=xgb_model, param_grid=param_grid_xgb, cv=stratified_cv, n_jobs=-1, verbose=2, scoring='recall')

# Fit the resampl data into XGBoost and train
grid_search_xgb.fit(X_train_resampled, Y_train_resampled_encoded)

Fitting 10 folds for each of 10800 candidates, totalling 108000 fits


E:\snakey\envs\raspberry\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:32:25] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='logloss', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     impor...
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'alpha': [0.0, 0.05, 0.1, 1.0],
                         'lambda': [0.0, 0.1, 1.0, 5.0, 10.0],
                         'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [3, 5, 10, 20],
                         'min_child_weight': [1, 3, 5],
                         'n_estimators': [100, 200, 225, 250, 300],
                         'subsample': [0.6, 0.8, 1.0]},
             scoring='recall', verbose=2)

In [64]:
# Print the classification report
best_xgb_model = grid_search_xgb.best_estimator_
Y_pred_xgb = best_xgb_model.predict(X_test)
print(classification_report(Y_test_encoded, Y_pred_xgb))

              precision    recall  f1-score   support

           0       0.88      0.95      0.91        22
           1       0.94      0.85      0.89        20

    accuracy                           0.90        42
   macro avg       0.91      0.90      0.90        42
weighted avg       0.91      0.90      0.90        42

